# Metaphor Search

Metaphor is a search engine fully designed to be used by LLMs. You can search and then get the contents for any page.

This notebook goes over how to use Metaphor search.

First, you need to set up the proper API keys and environment variables. Get 1000 free searches/month [here](https://platform.metaphor.systems/).

Then enter your API key as an environment variable.

In [1]:
import os

os.environ["METAPHOR_API_KEY"] = "..."

## Using their SDK

This is the newer and more supported way to use the Metaphor API - via their SDK

In [7]:
# !pip install metaphor-python

In [8]:
from metaphor_python import Metaphor

client = Metaphor(api_key=os.environ["METAPHOR_API_KEY"])

In [10]:
from langchain.agents import tool
from typing import List

In [11]:
@tool
def search(query: str):
    """Call search engine with a query."""
    return client.search(query)

@tool
def get_contents(ids: List[str]):
    """Get contents of a webpage.
    
    The ids passed in should be a list of ids as fetched from `search`.
    """
    return client.get_contents(ids)

@tool
def find_similar(url: str):
    """Get search results similar to a given URL.
    
    The url passed in should be a URL returned from `search`
    """
    return client.find_similar(url)

In [12]:
tools = [search, get_contents, find_similar]

### Use in an agent

In [13]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [16]:
from langchain.agents import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
system_message = SystemMessage(content="You are a web researcher who uses search engines to look up information.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [17]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.run("Find the hottest AI agent startups and what they do")



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'hottest AI agent startups'}`


SearchResponse(results=[Result(title='The AI Hot 75', url='https://www.nfx.com/post/generative-ai-hot-75-list', id='pb0vypILvIakocJicoqm4w', score=0.19051167368888855, published_date='2023-04-27', author='NFX', extract=None), Result(title='10 new AI unicorns flying high in 2022', url='https://venturebeat.com/ai/10-new-ai-unicorns-flying-high-in-2022/', id='i4k-4QKod9mMo2bi8m19bw', score=0.18090465664863586, published_date='2022-06-27', author='Sharon Goldman', extract=None), Result(title='The 13 most promising early-stage artificial-intelligence startups of 2022, according to VCs investing in companies building technical infrastructure and developing tools', url='https://www.businessinsider.com/13-most-promising-earlier-stage-ai-startups-this-year-2022-11', id='ZA52Zk-yVy9skxWZtxOSgg', score=0.1796971708536148, published_date='2022-11-17', author='Sindhu Sundar', extract=None), R

GetContentsResponse(contents=[DocumentContent(id='pb0vypILvIakocJicoqm4w', url='https://www.nfx.com/post/generative-ai-hot-75-list', title='The AI Hot 75', extract='<div><div><p><em>The hottest up &amp; comers in generative AI. Early-stage companies showing leading indicators of future greatness. Curated by NFX.</em></p>\n<p></p>\n<p>No doubt generative AI is extremely <a href="https://www.nfx.com/post/find-the-fast-moving-water">fast-moving water</a>. But while media hype focuses on incumbents and household names, we’re doubling down on the “AI underground.”</p>\n<p>The generative AI groundswell is disproportionately fueled by a dense network of exceptional, early-stage companies. We’ve seen it growing throughout San Francisco, in “Cerebral Valley” at NFX HQ, inside hacker houses, and via 100’s of meetings with up-and-coming AI founders.</p>\n<p>After we open-sourced our <a href="https://www.nfx.com/post/generative-ai-tech-market-map">Generative AI market map</a> a few months ago, we 

'Here are some of the hottest AI agent startups and what they do:\n\n1. The AI Hot 75: This list, curated by NFX, showcases early-stage companies in generative AI that show leading indicators of future greatness. These companies are at the forefront of the generative AI groundswell. [Read more](https://www.nfx.com/post/generative-ai-hot-75-list)\n\n2. 10 new AI unicorns flying high in 2022: This article highlights ten AI companies from the newest unicorn cohort that are soaring in 2022. One example is Abnormal Security, which uses behavioral AI to fight email attacks. [Read more](https://venturebeat.com/ai/10-new-ai-unicorns-flying-high-in-2022/)\n\n3. The 13 most promising early-stage artificial-intelligence startups of 2022: According to VCs investing in companies building technical infrastructure and developing tools, these 13 AI startups demonstrate the most potential. One example is OtterTune, a machine-learning-based platform to manage databases. [Read more](https://www.businessi

## Using the tool wrapper

This is the old way of using Metaphor - through our own in-house integration.

In [2]:
from langchain.utilities import MetaphorSearchAPIWrapper

In [3]:
search = MetaphorSearchAPIWrapper()

### Call the API
`results` takes in a Metaphor-optimized search query and a number of results (up to 500). It returns a list of results with title, url, author, and creation date.

In [4]:
search.results("The best blog post about AI safety is definitely this: ", 10)

[{'title': 'Core Views on AI Safety: When, Why, What, and How',
  'url': 'https://www.anthropic.com/index/core-views-on-ai-safety',
  'author': None,
  'published_date': '2023-03-08'},
 {'title': 'Extinction Risk from Artificial Intelligence',
  'url': 'https://aisafety.wordpress.com/',
  'author': None,
  'published_date': '2013-10-08'},
 {'title': 'The simple picture on AI safety - LessWrong',
  'url': 'https://www.lesswrong.com/posts/WhNxG4r774bK32GcH/the-simple-picture-on-ai-safety',
  'author': 'Alex Flint',
  'published_date': '2018-05-27'},
 {'title': 'No Time Like The Present For AI Safety Work',
  'url': 'https://slatestarcodex.com/2015/05/29/no-time-like-the-present-for-ai-safety-work/',
  'author': None,
  'published_date': '2015-05-29'},
 {'title': 'A plea for solutionism on AI safety - LessWrong',
  'url': 'https://www.lesswrong.com/posts/ASMX9ss3J5G3GZdok/a-plea-for-solutionism-on-ai-safety',
  'author': 'Jasoncrawford',
  'published_date': '2023-06-09'},
 {'title': 'The 

### Adding filters
We can also add filters to our search. 

include_domains: Optional[List[str]] - List of domains to include in the search. If specified, results will only come from these domains. Only one of include_domains and exclude_domains should be specified.

exclude_domains: Optional[List[str]] - List of domains to exclude in the search. If specified, results will only come from these domains. Only one of include_domains and exclude_domains should be specified.

start_crawl_date: Optional[str] - "Crawl date" refers to the date that Metaphor discovered a link, which is more granular and can be more useful than published date. If start_crawl_date is specified, results will only include links that were crawled after start_crawl_date. Must be specified in ISO 8601 format (YYYY-MM-DDTHH:MM:SSZ)

end_crawl_date: Optional[str] - "Crawl date" refers to the date that Metaphor discovered a link, which is more granular and can be more useful than published date. If endCrawlDate is specified, results will only include links that were crawled before end_crawl_date. Must be specified in ISO 8601 format (YYYY-MM-DDTHH:MM:SSZ)

start_published_date: Optional[str] - If specified, only links with a published date after start_published_date will be returned. Must be specified in ISO 8601 format (YYYY-MM-DDTHH:MM:SSZ). Note that for some links, we have no published date, and these links will be excluded from the results if start_published_date is specified.

end_published_date: Optional[str] - If specified, only links with a published date before end_published_date will be returned. Must be specified in ISO 8601 format (YYYY-MM-DDTHH:MM:SSZ). Note that for some links, we have no published date, and these links will be excluded from the results if end_published_date is specified.

See full docs [here](https://metaphorapi.readme.io/).

In [ ]:
search.results(
    "The best blog post about AI safety is definitely this: ",
    10,
    include_domains=["lesswrong.com"],
    start_published_date="2019-01-01",
)

### Use Metaphor as a tool
Metaphor can be used as a tool that gets URLs that other tools such as browsing tools.

In [ ]:
%pip install playwright
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)

async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

tools_by_name = {tool.name: tool for tool in tools}
print(tools_by_name.keys())
navigate_tool = tools_by_name["navigate_browser"]
extract_text = tools_by_name["extract_text"]

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import MetaphorSearchResults

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

metaphor_tool = MetaphorSearchResults(api_wrapper=search)

agent_chain = initialize_agent(
    [metaphor_tool, extract_text, navigate_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_chain.run(
    "find me an interesting tweet about AI safety using Metaphor, then tell me the first sentence in the post. Do not finish until able to retrieve the first sentence."
)